#### Libraries :

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

### 1. Data Gathering

In [2]:
df = pd.read_csv('spotify-2023.csv', encoding = 'ISO-8859-1')
df.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


In [3]:
#check column names
df.columns

Index(['track_name', 'artist(s)_name', 'artist_count', 'released_year',
       'released_month', 'released_day', 'in_spotify_playlists',
       'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts',
       'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm',
       'key', 'mode', 'danceability_%', 'valence_%', 'energy_%',
       'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%'],
      dtype='object')

In [4]:
#check shape of data
df.shape

(953, 24)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

#### 1.1 Handling Missing Values

In [6]:
#check missing values
df.isnull().sum()

track_name               0
artist(s)_name           0
artist_count             0
released_year            0
released_month           0
released_day             0
in_spotify_playlists     0
in_spotify_charts        0
streams                  0
in_apple_playlists       0
in_apple_charts          0
in_deezer_playlists      0
in_deezer_charts         0
in_shazam_charts        50
bpm                      0
key                     95
mode                     0
danceability_%           0
valence_%                0
energy_%                 0
acousticness_%           0
instrumentalness_%       0
liveness_%               0
speechiness_%            0
dtype: int64

In [7]:
#percentage missing value
missing_shazam_charts =(df['in_shazam_charts'].isnull().sum()/len(df))*100
missing_key =(df['key'].isnull().sum()/len(df))*100

print("% missing values in shazam charts column :", missing_shazam_charts)
print("% missing values in key column :", missing_key)

% missing values in shazam charts column : 5.246589716684155
% missing values in key column : 9.968520461699896


* shazam charts : since this represent ranking, missing values likely indicate that song didn't achieve a rank

* Our Strategy: We opted to handle the missing data in two ways:

    1.For interpretability, we created a copy of the column and filled missing values with a "Not Ranked" placeholder. This maintains clarity for anyone inspecting the dataset manually.

    2.For machine learning readiness, another copy of the column was made. Here, missing values are replaced with a numerical value (max_rank + 1). This ensures models receive purely numerical input without losing the information that these songs weren't ranked.
    
    
* This dual approach retains the integrity of the original data while making it usable for various analytical purposes.

In [8]:
#create a copy of human redable data
df['in_shazam_charts_readable'] = df['in_shazam_charts']
df['in_shazam_charts'] = df['in_shazam_charts'].replace({',':''},regex=True).astype(float)
df['in_shazam_charts_readable'].fillna('Not ranked', inplace=True)

In [9]:
#create a copy for ML processing
df['in_shazam_charts_ML'] = df['in_shazam_charts']
max_rank = df['in_shazam_charts_ML'].max()
df['in_shazam_charts_ML'].fillna(max_rank+1, inplace=True)
df.drop('in_shazam_charts',axis=1, inplace=True)

df['in_shazam_charts_ML'] = df['in_shazam_charts_ML'].astype(int)

* Key :

   * Analysis: This attribute signifies the musical key in which the song is composed. Assigning mean, median, or mode might be inappropriate and potentially misleading unless the missing values occur randomly without a discernable pattern.

   * Our Strategy: Given the nature of the 'key' attribute and considering its importance in our analysis, we have decided to remove the rows with missing key values to maintain the integrity and quality of our dataset. This approach ensures that we're working with complete data for each song in our subsequent analyses.

In [10]:
df = df.dropna(subset=['key'])
df.isnull().sum()

track_name                   0
artist(s)_name               0
artist_count                 0
released_year                0
released_month               0
released_day                 0
in_spotify_playlists         0
in_spotify_charts            0
streams                      0
in_apple_playlists           0
in_apple_charts              0
in_deezer_playlists          0
in_deezer_charts             0
bpm                          0
key                          0
mode                         0
danceability_%               0
valence_%                    0
energy_%                     0
acousticness_%               0
instrumentalness_%           0
liveness_%                   0
speechiness_%                0
in_shazam_charts_readable    0
in_shazam_charts_ML          0
dtype: int64

#### 1.2 Dealing with datatypes

In [11]:
#Inspect Current datatypes
df.dtypes

track_name                   object
artist(s)_name               object
artist_count                  int64
released_year                 int64
released_month                int64
released_day                  int64
in_spotify_playlists          int64
in_spotify_charts             int64
streams                      object
in_apple_playlists            int64
in_apple_charts               int64
in_deezer_playlists          object
in_deezer_charts              int64
bpm                           int64
key                          object
mode                         object
danceability_%                int64
valence_%                     int64
energy_%                      int64
acousticness_%                int64
instrumentalness_%            int64
liveness_%                    int64
speechiness_%                 int64
in_shazam_charts_readable    object
in_shazam_charts_ML           int32
dtype: object

In [12]:
# combine released year, released month and released day into one columns
df['released_date'] = pd.to_datetime(df['released_year'].astype(str)+'-'+ 
                                     df['released_month'].astype(str)+'-'+
                                     df['released_day'].astype(str))
df['released_date'].head()

0   2023-07-14
1   2023-03-23
2   2023-06-30
3   2019-08-23
4   2023-05-18
Name: released_date, dtype: datetime64[ns]

In [17]:
#convert 'streams' and 'in_deezer_playlists' to appropriate datatype
df['streams'] = pd.to_numeric(df['streams'], errors ='coerce')
df['in_deezer_playlists'] = pd.to_numeric(df['in_deezer_playlists'],errors='coerce')

In [19]:
print(df['streams'].dtype)
print(df['in_deezer_playlists'].dtype)

float64
float64


In [26]:
#missing values in 'streams' and 'in_deezer_playlists'
print("Number of missing values in streams :",df['streams'].isnull().sum())
print("Number of missing values in deezer playlists :", df['in_deezer_playlists'].isnull().sum())

Number of missing values in streams : 1
Number of missing values in deezer playlists : 75


In [29]:
df[df['streams'].isna()]

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,in_shazam_charts_readable,in_shazam_charts_ML,released_date
574,Love Grows (Where My Rosemary Goes),Edison Lighthouse,1,1970,1,1,2877,0,NaN,16,...,53,75,69,7,0,17,3,0,0,1970-01-01


In [32]:
#remove the row from dataframe
df.dropna(subset=['streams'],inplace=True)

#verify if its dropped
df[df['streams'].isna()]

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,in_shazam_charts_readable,in_shazam_charts_ML,released_date


In [34]:
df[df['in_deezer_playlists'].isna()].head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,in_shazam_charts_readable,in_shazam_charts_ML,released_date
48,Starboy,"The Weeknd, Daft Punk",2,2016,9,21,29536,79,2.565530e+09,281,...,68,49,59,16,0,13,28,140,140,2016-09-21
54,Another Love,Tom Odell,1,2012,10,15,18371,83,1.813674e+09,250,...,45,13,54,70,0,9,4,Not ranked,1452,2012-10-15
55,Blinding Lights,The Weeknd,1,2019,11,29,43899,69,3.703895e+09,672,...,50,38,80,0,0,9,7,Not ranked,1452,2019-11-29
65,Yellow,Chris Molitor,1,1999,1,1,31358,43,1.755214e+09,196,...,43,28,66,0,0,23,3,0,0,1999-01-01
73,Sweater Weather,The Neighbourhood,1,2012,5,14,16413,61,2.282771e+09,166,...,61,41,81,5,2,10,3,Not ranked,1452,2012-05-14


In [35]:
df['in_deezer_playlists'].fillna(0,inplace=True)

In [37]:
#revised datatypes
df.dtypes

track_name                           object
artist(s)_name                       object
artist_count                          int64
released_year                         int64
released_month                        int64
released_day                          int64
in_spotify_playlists                  int64
in_spotify_charts                     int64
streams                             float64
in_apple_playlists                    int64
in_apple_charts                       int64
in_deezer_playlists                 float64
in_deezer_charts                      int64
bpm                                   int64
key                                  object
mode                                 object
danceability_%                        int64
valence_%                             int64
energy_%                              int64
acousticness_%                        int64
instrumentalness_%                    int64
liveness_%                            int64
speechiness_%                   

In [38]:
#encoding of 'mode' and 'key' columns
#one hot encoding the 'mode' column keeping the original column untouched

mode_encoded = pd.get_dummies(df['mode'], prefix='mode')
df = pd.concat([df,mode_encoded], axis=1)

In [39]:
#label encoding 'key' column while keeping the original untouched
label_encoder = LabelEncoder()
df['key_encoded'] = label_encoder.fit_transform(df['key'].astype(str))

In [40]:
df[['mode',*mode_encoded.columns, 'key','key_encoded']]

,mode,mode_Major,mode_Minor,key,key_encoded
0,Major,1,0,B,2
1,Major,1,0,C#,3
2,Major,1,0,F,7
3,Major,1,0,A,0
4,Minor,0,1,A,0
...,...,...,...,...,...
948,Major,1,0,A,0
949,Major,1,0,F#,8
950,Major,1,0,C#,3
951,Major,1,0,C#,3


In [45]:
percentage_columns = df.iloc[:,-7:]
percentage_columns

,speechiness_%,in_shazam_charts_readable,in_shazam_charts_ML,released_date,mode_Major,mode_Minor,key_encoded
0,4,826,826,2023-07-14,1,0,2
1,4,382,382,2023-03-23,1,0,3
2,6,949,949,2023-06-30,1,0,7
3,15,548,548,2019-08-23,1,0,0
4,6,425,425,2023-05-18,0,1,0
...,...,...,...,...,...,...,...
948,3,0,0,2022-11-03,1,0,0
949,6,0,0,2022-10-21,1,0,8
950,6,0,0,2022-11-03,1,0,3
951,5,0,0,2022-10-20,1,0,3
